In [1]:
import dotenv
import os
from playwright.async_api import async_playwright
import time
import random
import json

In [2]:
# load environment variables
env = dotenv.load_dotenv()

amazon_home = "https://www.amazon.com/"

In [3]:
# Launch Browser
async def start_browser():
    p = await async_playwright().start()
    browser = await p.chromium.launch(
        headless=False
        # executable_path="/Users/sahilhadke/Library/Caches/ms-playwright/firefox-1471/firefox/firefox"
    )
    context = await browser.new_context()
    page = await context.new_page()
    return p, browser, context, page

# Execute this block to start the browser
p, browser, context, page = await start_browser()
await page.goto(amazon_home)

<Response url='https://www.amazon.com/' request=<Request url='https://www.amazon.com/' method='GET'>>

In [4]:
# Login Manually
async def login():
    # click on sign in button
    sign_in_button = await page.wait_for_selector("//a[@data-nav-role='signin']")
    await sign_in_button.click()

    # enter email
    email_input = await page.wait_for_selector("input[type='email']")
    await email_input.fill(os.getenv("AMAZON_EMAIL"))

    # click on continue
    continue_button = await page.wait_for_selector("//input[@id='continue']")

    # click on continue
    await continue_button.click()

    # enter password
    password_input = await page.wait_for_selector("input[type='password']")
    await password_input.fill(os.getenv("AMAZON_PASSWORD"))

    # click on sign in
    sign_in_button = await page.wait_for_selector("//input[@id='signInSubmit']")
    await sign_in_button.click()

# Execute this block to login
await login()

In [5]:
# read all json files from directory ./output
json_files = [f for f in os.listdir('/Users/sahilhadke/Desktop/PROJECTS/amazon-review-scraper/output') if f.endswith('.json')]
for json_file in json_files:
    with open(f"./output/{json_file}", 'r') as file:
        data = json.load(file)
        
        for product in data['products']:
            url_to_scrape = product['url'] 

            if "total_review_count" in product:
                continue

            # go to url
            await page.goto(url_to_scrape)

            # scrape number of ratings
            # Scroll to Reviews Section
            for _ in range(20):
                await page.evaluate("window.scrollBy(0, 1000)")
                time.sleep(0.5)

                review_more_link = await page.query_selector("//a[@data-hook='see-all-reviews-link-foot']")
                if review_more_link:
                    await review_more_link.scroll_into_view_if_needed()
                    await review_more_link.click()
                    break

            time.sleep(2)

            # total reivew count (//span[@data-hook="total-review-count"])[1]
            total_review_count = await page.query_selector("//div[@data-hook='total-review-count']//span")
            if not total_review_count:
                continue
            total_review_count = await total_review_count.inner_text()
            total_review_count = total_review_count.split()[0]
            print(total_review_count)

            # update the json file
            product['total_review_count'] = total_review_count

            # get arialabel text
            for i in range(1, 6):
                stars_xpath = f"//ul[@id='histogramTable']//li[{i}]//a"
                star = await page.query_selector(stars_xpath)
                try:
                    arialabel = await star.get_attribute("aria-label")
                    # start 
                    star_number = arialabel.split(" ")[0]
                    percentage = ""
                    for elem in arialabel.split(" "):
                        if '%' in elem:
                            percentage = elem
                            break
                    product[f"{star_number}_star_rating_percentage"] = percentage
                except:
                    product[f"{6-i}_star_rating_percentage"] = "0%"
                
                

                # update product
                
            


            with open(f"./output/{json_file}", 'w') as file:
                json.dump(data, file, indent=4)

            time.sleep(random.randint(1, 5))
            
            

298
1,446
315
198
736
609
339
280
34
53
333
124
27
294
98
1,042
1,042
34
608
329
1,135
450
423
210
329
105
83
6
329
39
452
213
478
15
6


In [14]:
# close
await browser.close()